In [48]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from joblib import dump, load

from collections import Counter
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=10**5, n_features=10**2,
                           n_informative=3, n_redundant=0,
                           random_state=0, shuffle=False)
X = (X).astype('int8')
X = (X>0).astype('int8')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
gini = 2*roc_auc_score(y_train,clf.predict(X_train))-1
print('train gini: ',gini)

gini = 2*roc_auc_score(y_test,clf.predict(X_test))-1
print('test gini: ',gini)
dump(clf, 'sklearn_dt.joblib')

train gini:  0.2753588460903724
test gini:  0.28091140982027385


['sklearn_dt.joblib']

In [50]:
#%timeit clf.predict(X_train)

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

clf_rf = RandomForestClassifier()
clf_rf.fit(X_train,y_train)

gini = 2*roc_auc_score(y_train,clf_rf.predict(X_train))-1
print('train gini: ',gini)

gini = 2*roc_auc_score(y_test,clf_rf.predict(X_test))-1
print('test gini: ',gini)

from joblib import dump, load
dump(clf_rf, 'sklearn_rf.joblib')

train gini:  0.2753588460903724
test gini:  0.28091140982027385


['sklearn_rf.joblib']

In [52]:
#%timeit clf_rf.predict(X_train)

Sklearn implementation stores irrelevant information from our perspective. Each node contains the following:
- id for the split (int64)
- id for the parent node (int64)
- impurity or gain of the split (float64)

In [53]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None

class Ptree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None,random_state=34):
        self.min_samples_split=min_samples_split
        self.max_depth=max_depth
        self.n_features=n_features
        self.root=None
        self._is_fitted = False
        self._random_state = random_state
        self.shuffled_features = None
        self.leaf_counter = 0

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)

        #self.shuffled_features = sklearn.utils.shuffle(self.n_features,random_state=self._random_state)
        self.root = self._grow_tree(X, y)
        self._is_fitted = True

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            leaf_value = self._most_common_label(y)
            self.leaf_counter = self.leaf_counter+1
            return Node(value=leaf_value)
        
        # find the best split in the data
        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_gain = -1
        best_feature_idx = None
        for feat_idx in feat_idxs:

            # check feature has variance
            X_col = X[:,feat_idx]
            no_unique_vals = np.unique(X_col)
            if (len(no_unique_vals)==1):
                continue

            # if it has variance then check the information gain
            gain = self._information_gain(y, X_col)
            if gain>best_gain:
                best_gain=gain
                best_feature_idx = feat_idx
        
        # if there are no vars with variance then return as a leaf
        if best_feature_idx is None:
            leaf_value = self._most_common_label(y)
            self.leaf_counter = self.leaf_counter+1
            return Node(value=leaf_value)
        
        left_idxs, right_idxs = self._split(X[:, best_feature_idx])
        
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature_idx, 1, left, right)


    def _best_feature(self, X, y, feat_idxs):
        best_gain = -1
        best_feature_idx = None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            gain = self._information_gain(y, X_column)
            if gain > best_gain:
                best_gain = gain
                best_feature_idx = feat_idx

        return best_feature_idx


    def _information_gain(self, y, X_column):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = (X_column==0).flatten().nonzero()[0], (X_column==1).flatten().nonzero()[0]
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column):
        left_idxs = (X_column==0).flatten().nonzero()[0]
        right_idxs = (X_column==1).flatten().nonzero()[0]
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])


    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] == 0:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
clf_cdt = Ptree()
clf_cdt.fit(X_train,y_train)

print('y_train mean:',np.mean(y_train))

y_train mean: 0.5004666666666666


In [54]:
#%timeit clf_cdt.predict(X)

In [55]:
gini = 2*roc_auc_score(y_train,clf_cdt.predict(X_train))-1
print('train gini: ',gini)

gini = 2*roc_auc_score(y_test,clf_cdt.predict(X_test))-1
print('test gini: ',gini)
dump(clf_cdt, 'sklearn_custom_dt.joblib')

train gini:  0.2753588460903724
test gini:  0.28091140982027385


['sklearn_custom_dt.joblib']

In [56]:
class PRandF:
    def __init__(self,n_estimators=32,max_depth=None,random_state=34):
        self._n_estimators = n_estimators
        self._random_state = random_state
        self._nclasses = None

        #random_state = np.arange(self._n_estimators)
        self._estimators = [Ptree(random_state=i) for i in range(self._n_estimators)]
        
    def fit(self,X,y):
        self._nclasses = len(np.unique(y))
        self._classes = np.arange(self._nclasses)
        for estimator in self._estimators:
            estimator.fit(X,y)

    def predict(self,X):
        # for each estimator get the predicted class
        # then create an empty zero array and fill the ith column with the class
        predictions = np.stack([estimator.predict(X) for estimator in self._estimators])
        predictions = np.transpose(predictions)

        all_probs = list()
        for obs_predictions in predictions:
            obs_class_predictions = np.zeros(self._nclasses)
            for obs_prediction in obs_predictions:
                obs_class_predictions[obs_prediction] = obs_class_predictions[obs_prediction] + 1
            obs_class_predictions = obs_class_predictions/sum(obs_class_predictions)
            all_probs.append(obs_class_predictions)

        probabilities = np.stack(all_probs)
        predictions = probabilities.argmax(axis=1)
        
        return predictions

clf_crf = PRandF(n_estimators=32)
clf_crf.fit(X_train,y_train)

In [57]:
#%timeit clf_crf.predict(X_test)

In [58]:
gini = 2*roc_auc_score(y_train,clf_crf.predict(X_train))-1
print('train gini: ',gini)

gini = 2*roc_auc_score(y_test,clf_crf.predict(X_test))-1
print('test gini: ',gini)
dump(clf_crf, 'sklearn_custom_rf.joblib')

train gini:  0.2753588460903724
test gini:  0.28091140982027385


['sklearn_custom_rf.joblib']

In [59]:
39/250

0.156

In [60]:
10*0.15

1.5